In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import ADASYN
from collections import Counter

from numpy.random import seed
seed(7)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [2]:
df = pd.read_csv("combined_filtered.csv") 
df.head(n=5) 

,Unnamed: 0,_time,GreenRocket,Door1,Door2,LoadCell_R02,Potentiometer_R02,LoadCell_R03,Potentiometer_R03,LoadCell_R04,Potentiometer_R04,Anomally,Success
0,0,2023-09-19T14:21:53.393Z,1,0,0,1857,11866,1693,2390,2640,11855,0,1
1,1,2023-09-19T14:21:53.485Z,1,0,0,1862,11896,1679,2396,2643,11908,0,1
2,2,2023-09-19T14:21:53.582Z,1,0,0,1867,11957,1669,2379,2659,11939,0,1
3,3,2023-09-19T14:21:53.687Z,1,0,0,1886,11955,1699,2403,2612,11932,0,1
4,4,2023-09-19T14:21:53.793Z,1,0,0,1887,11966,1670,2388,2630,11931,0,1


In [3]:
df['Success'] = df['Success'].replace({1: 0, 0: 1})
df

,Unnamed: 0,_time,GreenRocket,Door1,Door2,LoadCell_R02,Potentiometer_R02,LoadCell_R03,Potentiometer_R03,LoadCell_R04,Potentiometer_R04,Anomally,Success
0,0,2023-09-19T14:21:53.393Z,1,0,0,1857,11866,1693,2390,2640,11855,0,0
1,1,2023-09-19T14:21:53.485Z,1,0,0,1862,11896,1679,2396,2643,11908,0,0
2,2,2023-09-19T14:21:53.582Z,1,0,0,1867,11957,1669,2379,2659,11939,0,0
3,3,2023-09-19T14:21:53.687Z,1,0,0,1886,11955,1699,2403,2612,11932,0,0
4,4,2023-09-19T14:21:53.793Z,1,0,0,1887,11966,1670,2388,2630,11931,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216029,216029,2023-09-19T20:21:59.584Z,0,0,0,1888,2061,1676,2382,2604,11801,1,1
216030,216030,2023-09-19T20:21:59.686Z,0,0,0,1854,2061,1686,2404,2776,11878,1,1
216031,216031,2023-09-19T20:21:59.785Z,0,0,0,1859,2073,1685,2403,3218,11917,1,1
216032,216032,2023-09-19T20:21:59.887Z,0,0,0,1860,2067,1655,2385,2176,11904,1,1


In [4]:
df['Success'].value_counts()

0    212296
1      3738
Name: Success, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216034 entries, 0 to 216033
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         216034 non-null  int64 
 1   _time              216034 non-null  object
 2   GreenRocket        216034 non-null  int64 
 3   Door1              216034 non-null  int64 
 4   Door2              216034 non-null  int64 
 5   LoadCell_R02       216034 non-null  int64 
 6   Potentiometer_R02  216034 non-null  int64 
 7   LoadCell_R03       216034 non-null  int64 
 8   Potentiometer_R03  216034 non-null  int64 
 9   LoadCell_R04       216034 non-null  int64 
 10  Potentiometer_R04  216034 non-null  int64 
 11  Anomally           216034 non-null  int64 
 12  Success            216034 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 21.4+ MB


In [6]:
# Remove time column
df = df.drop(['_time','Unnamed: 0'], axis=1)


In [7]:
X = df.drop(['Success','Anomally'], axis=1)
y = df['Success']

In [8]:
X
X=X.fillna(0)
X

,GreenRocket,Door1,Door2,LoadCell_R02,Potentiometer_R02,LoadCell_R03,Potentiometer_R03,LoadCell_R04,Potentiometer_R04
0,1,0,0,1857,11866,1693,2390,2640,11855
1,1,0,0,1862,11896,1679,2396,2643,11908
2,1,0,0,1867,11957,1669,2379,2659,11939
3,1,0,0,1886,11955,1699,2403,2612,11932
4,1,0,0,1887,11966,1670,2388,2630,11931
...,...,...,...,...,...,...,...,...,...
216029,0,0,0,1888,2061,1676,2382,2604,11801
216030,0,0,0,1854,2061,1686,2404,2776,11878
216031,0,0,0,1859,2073,1685,2403,3218,11917
216032,0,0,0,1860,2067,1655,2385,2176,11904


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((172827, 9), (43207, 9))

In [10]:
# Feature scaling

In [11]:
cols = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])


In [12]:
# define model
model = XGBClassifier()

In [14]:
# Define SMOTE-Tomek Links
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
# Define pipeline
pipeline=Pipeline(steps=[('r', resample), ('m', model)])

In [15]:
clf_0 = pipeline.fit(X_train, y_train)


In [16]:
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))


0.9918994607355289


In [17]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_0)

              precision    recall  f1-score   support

     class 0       1.00      0.99      1.00     42477
     class 1       0.72      0.85      0.78       730

    accuracy                           0.99     43207
   macro avg       0.86      0.92      0.89     43207
weighted avg       0.99      0.99      0.99     43207



array([[42233,   244],
       [  106,   624]])

In [18]:
#enn

In [19]:
# define model
model = XGBClassifier()

#Define SMOTE-ENN
resample=SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='all'))
#Define pipeline
pipeline=Pipeline(steps=[('r', resample), ('m', model)])

In [20]:
pipeline

Pipeline(steps=[('r',
                 SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='all'))),
                ('m',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [21]:
clf_2 = pipeline.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))

0.9872937255537297


In [22]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

              precision    recall  f1-score   support

     class 0       1.00      0.99      0.99     42477
     class 1       0.58      0.94      0.71       730

    accuracy                           0.99     43207
   macro avg       0.79      0.96      0.85     43207
weighted avg       0.99      0.99      0.99     43207



array([[41972,   505],
       [   44,   686]])

In [23]:
#ADASYN
ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X, y)

In [24]:
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 212296, 1: 211836})


In [25]:
model = XGBClassifier()

#Define ADASYN
resample=ADASYN(random_state=42)
#Define pipeline
pipeline=Pipeline(steps=[('r', resample), ('m', model)])

In [26]:
clf_2 = pipeline.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))

0.9913671395838637


In [27]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

              precision    recall  f1-score   support

     class 0       1.00      0.99      1.00     42477
     class 1       0.70      0.87      0.77       730

    accuracy                           0.99     43207
   macro avg       0.85      0.93      0.88     43207
weighted avg       0.99      0.99      0.99     43207



array([[42200,   277],
       [   96,   634]])